In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Loading the Data

In [2]:
flights = pd.read_csv('nycflights.csv')

In [3]:
flights.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,1/1/2013 5:00
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,1/1/2013 5:00
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,1/1/2013 5:00
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,1/1/2013 5:00
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,1/1/2013 6:00


In [ ]:
flights.info()

We load the data, take a look at the head, and the descriptions.

It seems that dep_time, dep_delay, arr_time, arr_delay, and air_time were all converted into floats. These are supposed to represent times where '1455' represent the time '14:55'. There should not be any decimal values. The fact that decimals are appearing means that something happened in the import that made python believe they were floating point numbers.

In [ ]:
flights.tail()

An inspection of the tail shows that there are missing values in dep_time and the other columns. Because python represents missing numeric values with NaN, they were convered to floating point numbers.

We will reload the flights csv file, this time specifying the offending columns as 'object' type. (It's a little weird, but we do not use str type because str uses NumPy's str type which is different from regular Python strings. To use Python strings, Pandas uses 'object' type)

## Loading the data again and doing a little bit of clean up

In [ ]:
flights = pd.read_csv('nycflights.csv', dtype = {'dep_time' : object, 
                                                 'dep_delay': object, 
                                                 'arr_time' : object, 
                                                 'arr_delay': object, 
                                                 'air_time' : object})

In [ ]:
flights.head() # the dep_time column no longer has any decimals in it.

In [ ]:
flights.tail(10) # the missing values are still preserved

The next thing I notice is that the dates are split across three columns: year, month, day.

I will use Python's date functions to combine the three values into a single date value

In [ ]:
flights['date'] = pd.to_datetime({'year':flights.year, 'month':flights.month, 'day':flights.day})

In [ ]:
flights.head() # we have now appeneded a new column that shows the date

With the times expressed as they are, we can't calculate the real time difference between columns. The time between '600' and '545' is really only 15 minutes, but the math will make the wrong calculation of 55. Thus, we need to split our text into hour and minute and make them into date-time values.

In [ ]:
# we filter the dep_time to just non_null values
# I will extract the minute values by treating the series as a string and pulling the last two characters
# we get the hour by taking everything but the last two characters
dep_min  = flights.dep_time[ flights.dep_time.notnull() ].astype(str).str[-2:  ]
dep_hour = flights.dep_time[ flights.dep_time.notnull() ].astype(str).str[  :-2]
print(dep_hour.head())
print(dep_min.head())

In [ ]:
dep_date_time = pd.to_datetime({'year':flights.year, 'month':flights.month, 'day':flights.day, 
                                'hour': dep_hour, 'minute': dep_min})

In [ ]:
print(dep_date_time.head(10))
print(dep_date_time.tail())

I'd like to do the same kind of conversion for all of the other time columns. 

Instead of copying and pasting the code and modifying it for each column, I'm going to build a function.

In [ ]:
def convert_time(column_name):
    the_min = flights[column_name][ flights[column_name].notnull() ].astype(str).str[-2:  ]
    the_hr  = flights[column_name][ flights[column_name].notnull() ].astype(str).str[  :-2]
    date_time = pd.to_datetime({'year':flights.year, 'month':flights.month, 'day':flights.day, 
                                'hour': the_hr, 'minute': the_min})
    # the magic of indexes allow us to combine flights.year which has all the rows
    # and the_hr which only exists for rows where dep_time is not NaN
    return date_time

In [ ]:
# let's test out the function
dep_date_time = convert_time('dep_time')

In [ ]:
print(dep_date_time.head(10))
print(dep_date_time.tail(10))

In [ ]:
# seems to work!
# let's go ahead and use it:
# dep_date_time       = convert_time('dep_time')
sched_dep_date_time = convert_time('sched_dep_time')
arr_date_time       = convert_time('arr_time')
sched_arr_date_time = convert_time('sched_arr_time')

In [ ]:
# pd.concat creates a new dataframe by joining several pandas objects together
datetimes = pd.concat({'dep_date_time': dep_date_time,
           'sched_dep_date_time': sched_dep_date_time, 
           'arr_date_time':arr_date_time, 
           'sched_arr_date_time':sched_arr_date_time}, axis = 1)
datetimes.head()

In [ ]:
flights = flights.join(datetimes)  # we use the .join() function

In [ ]:
flights.head()

Let's go ahead and drop the year, month, and day columns as we don't need them anymore.

In [ ]:
flights = flights.drop(['year','month','day'], axis = 1)

In [ ]:
flights.head()

If we need to get the month or day, we can extract them using the .dt. object attributes

In [ ]:
flights.date.dt.month.head()

Now that we've got the data in and cleaned up, let's do some Basic Exploratory Analysis

## Let's start with some basics:

### How many unique carriers are there? How many flights for each carrier?

In [ ]:
flights.carrier.unique()

In [ ]:
flights.carrier.nunique()

In [ ]:
carrier_counts = flights.carrier.value_counts()
carrier_counts

In [ ]:
airlines = pd.read_csv('airlines.csv', index_col = 'carrier')
airlines

In [ ]:
airlines_count = airlines.join(carrier_counts)
airlines_count

In [ ]:
airlines_count.rename(index = str, columns = {"name": "name", "carrier": "count" })

### What about origination airports?

In [ ]:
flights.origin.unique()

In [ ]:
flights.origin.value_counts()  # it appears that the three airports in the new york area are used with similar frequency

In [ ]:
# alternative method using .groupby(), which is similar to dplyr::groupby
flights.groupby('origin').sched_dep_date_time.count()

In [ ]:
# if we perform a count on the dep_date_time column, it does not include NaNs.
flights.groupby('origin').dep_date_time.count()

### Destination airports?

In [ ]:
flights.dest.unique()

In [ ]:
flights.dest.nunique()  # 105 destinations

In [ ]:
flights.dest.value_counts().head(10)  # a quick peek at the top 10 destinations

### Which month is most busy?

In [ ]:
flights.dep_date_time.dt.month.value_counts()

In [ ]:
flights.dep_date_time.dt.month.value_counts().sort_index().plot(kind = 'bar', color = 'dodgerblue')
plt.title('Number of Flights Each Month')
plt.show()

### What time of day is most busy?

In [ ]:
# using groupby instead of value_counts
flights.groupby(dep_date_time.dt.hour).date.count()

In [ ]:
flights.groupby(dep_date_time.dt.hour).date.count().plot(kind = 'bar', color = 'dodgerblue')
plt.title('Flights Each Hour')
plt.show()